In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import stats
from helper import *
from scipy.stats.mstats import gmean
#Importation of all the packages
from datetime import datetime
import math
import json
import zipfile  
import ssl
from scipy import stats

#To dowload data
import requests
import io
import gzip

#To create the mapchart
import iso3166
import plotly
from iso3166 import countries
import plotly.graph_objects as go

In [ ]:
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
pageview_df_imp_country = pageview_df[["date"] + list(get_country_dict().values())].set_index('date')
pageview_df_imp_country.head()
country_dict = get_country_dict()

In [ ]:
#Normalisation different Mean/Max;Min; Correlation pageviews avec covid cases

#Population per country
#Population per languages


In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
death_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_deaths.csv?raw=true" # Make sure the url is the raw version of the file on GitHub
cases_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_cases.csv?raw=true"

# Reading the downloaded content and turning it into a pandas dataframe
deaths = pd.read_csv(death_url,index_col=0)
cases = pd.read_csv(cases_url,index_col=0)
deaths= deaths.interpolate(method ='linear', limit_direction ='forward') 
cases = cases.interpolate(method ='linear', limit_direction ='forward') 
deaths = deaths.fillna(0)
cases = cases.fillna(0)
#Keep only values until July
deaths = deaths[deaths.index < '2020-08-31']
cases = cases[cases.index < '2020-08-31']


In [ ]:
df_pageviews_pearson = pageview_df_imp_country.loc[pageview_df_imp_country.index < '2020-08-31']
df_pageviews_pearson = df_pageviews_pearson.loc[df_pageviews_pearson.index >= '2020-01-22']
df_pageviews_pearson = df_pageviews_pearson.interpolate(method ='linear', limit_direction ='forward') 
df_pageviews_pearson = df_pageviews_pearson.fillna(0)

In [ ]:
pearson_deaths = pd.DataFrame({"Pearson correlation":["Correlation coefficient:","P-value:"]})

for country in country_dict.items():
    corr, p_value = stats.pearsonr(df_pageviews_pearson[country[1]], deaths[country[0]])
    output_df = pd.DataFrame({str(country[0]):[corr, p_value] })
    pearson_deaths = pd.concat([pearson_deaths, output_df], axis=1)
pearson_deaths

In [ ]:
pearson_cases = pd.DataFrame({"Pearson correlation":["Correlation coefficient:","P-value:"]})

for country in country_dict.items():
    corr, p_value = stats.pearsonr(df_pageviews_pearson[country[1]], cases[country[0]])
    output_df = pd.DataFrame({str(country[0]):[corr, p_value] })
    pearson_cases = pd.concat([pearson_cases, output_df], axis=1)
pearson_cases

In [ ]:
# create a new figure
plt.figure()

# use the scatter() function to plot the points,
# with red circles of size 10
plt.scatter(pearson_deaths.loc[1][1:], pearson_deaths.loc[0][1:], color="red", s=10)

# add a title
plt.title("Scatter Plot pearson death and pageviews")

# add x and y axis labels
plt.xlabel("Corr coeff")
plt.ylabel("P-value")

# show the plot
plt.show()

In [ ]:
pearson_deathsT = pearson_deaths.transpose()
pearson_deathsT = pearson_deathsT.drop(index= 'Pearson correlation', axis=0)
pearson_deathsT = pearson_deathsT.rename(columns= {0: 'Correlation coefficient', 1: 'P-value'})
# create a new figure
plt.figure(figsize=(40, 10))

# use the scatter() function to plot the points,
# with red circles of size 10
plt.bar(pearson_deathsT.index.values, pearson_deathsT['Correlation coefficient'].values, color="red")

# add a title
plt.title("Scatter Plot pearson death and pageviews")

# add x and y axis labels
plt.xlabel("Countries")
plt.ylabel("Corr coeff (deaths/pageviews)")

# show the plot
plt.show()

### Loading population by countries

In [ ]:
COUNTRY_OWN_LANG_POP = {"Italy" : "it", "Russian Federation": "ru", "China": "zh", "Albania": "sq", 
 "Bangladesh": "bn", "Botswana": "tn", "Cambodia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
 "Malaysia": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovak Republic": "sk", "Slovenia": "sl", "Turkiye": "tr",
 "Vietnam": "vi", "Bulgaria": "bg", "Czechia": "cs", "Denmark": "da", "Georgia": "ka", "Germany": "de", 
 "Hungary": "hu", "Iceland": "is", "Japan": "ja", "Kazakhstan": "kk", "Korea, Rep.": "ko", "Kyrgyz Republic": 'ky', "Netherlands": "nl", "Poland": "pl", 
 "Romania": "ro", "Tajikistan": "tg", "Thailand": "th", "Azerbaijan": "az", "Mongolia": "mn"}

#changed_name_countries = {"Russian Federation": "Russia", "Slovak Republic": "Slovakia", "Turkiye": "Turkey",
# "Korea, Rep.": "South Korea", "Kyrgyz Republic": 'Kyrgyzstan'}

population_df = pd.read_csv("Population_countries.csv")
population_df = population_df[["Country Name", "2020"]]
population_df = population_df.set_index("Country Name")
population_df = population_df.transpose()
population_df = population_df[list(COUNTRY_OWN_LANG_POP.keys())]
population_df = population_df.rename(columns= COUNTRY_OWN_LANG_POP)
population_df = population_df.transpose()
population_df.head()

In [ ]:
population_df['2020']

## Check different metrics for pageviews, death and cases

### 1. Pageviews

In [ ]:

pageviews_amean = (df_pageviews_pearson.mean()/population_df['2020']) * 100000
pageviews_max = (df_pageviews_pearson.max()/population_df['2020']) * 100000
pageviews_min = (df_pageviews_pearson.min()/population_df['2020']) * 100000


In [ ]:

# create a new figure
plt.figure(figsize=(12, 8))
# use the scatter() function to plot the points,
# with red circles of size 10
plt.scatter(pageviews_max.index, pageviews_max.values, color="red", s= 15)
plt.scatter(pageviews_min.index, pageviews_min.values, color="blue", s= 15)
plt.scatter(pageviews_amean.index, pageviews_amean.values, color="green", s= 15)
# add a title
plt.title("Scatter Plot pageviews metrics per countries")

# add x and y axis labels
plt.xlabel("Languages")
plt.ylabel("Pageviews")
plt.legend(["Max", "Min", "Arithmetic Mean"])

# show the plot
plt.show()

### 1. Covid-19 dataset
We have to select the variable between covid cases and covid deaths that create the most variability between the countries. Moreover, we also have to choose a represenative metric such as mean, min or max. 

Let's first plot the covid deaths for all the countries per 100 000 people, looking at the mean,min,max. 

In [ ]:
deaths_lang_code = deaths.rename(columns= country_dict)[country_dict.values()]
deaths_amean = (deaths_lang_code.mean()/population_df['2020']) * 100000
deaths_max = (deaths_lang_code.max()/population_df['2020']) * 100000
deaths_min = (deaths_lang_code.min()/population_df['2020']) * 100000


In [ ]:
# create a new figure
plt.figure(figsize=(12, 8))
# use the scatter() function to plot the points,
# with red circles of size 10
plt.scatter(deaths_max.index, deaths_max.values, color="red", s= 15)
plt.scatter(deaths_min.index, deaths_min.values, color="blue", s= 15)
plt.scatter(deaths_amean.index, deaths_amean.values, color="green", s= 15)
# add a title
plt.title("Scatter Plot pageviews metrics per countries")

# add x and y axis labels
plt.xlabel("Languages")
plt.ylabel("Deaths for 100000 pop")
plt.legend(["Max", "Min", "Arithmetic Mean"])

# show the plot
plt.show()


Now, let's look at covid cases for 100 000 people per country

In [ ]:
cases_lang_code = cases.rename(columns= country_dict)[country_dict.values()]
cases_amean = (cases_lang_code.mean()/population_df['2020']) * 100000
cases_max = (cases_lang_code.max()/population_df['2020']) * 100000
cases_min = (cases_lang_code.min()/population_df['2020']) * 100000

Looking at both plots, we see that the max value gives much more variance between the countries than mean or min, therefore this metric is chosen for the clustering. Moreover, we also see that covid cases also shows more variance than covid deaths between the countries, therefore the max of covid cases is chosen for the clustering of countries. 

In [356]:
data_folder = 'data_2/'
df_trust_gov = pd.read_csv(data_folder+'share-who-trust-government.csv.zip') 
df_trust_science = pd.read_csv(data_folder+'share-people-trust-science.csv.zip')
df_trust_journalists = pd.read_csv(data_folder+'share-people-trust-journalists-country.csv.zip')


In [357]:
COUNTRY_OWN_LANG_TRUST_JOURN = {"Italy" : "it", "Russia": "ru", "China": "zh", "Albania": "sq", 
"Bangladesh": "bn", "Cambodia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
 "Malaysia": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovakia": "sk", "Slovenia": "sl", "Turkey": "tr",
 "Vietnam": "vi", "Bulgaria": "bg", "Czechia": "cs", "Denmark": "da", "Georgia": "ka", "Germany": "de", 
 "Hungary": "hu", "Japan": "ja", "Kazakhstan": "kk", "South Korea": "ko", "Kyrgyzstan": 'ky', "Netherlands": "nl", "Poland": "pl", 
 "Romania": "ro", "Tajikistan": "tg", "Thailand": "th", "Mongolia": "mn"}

#df_trust_gov = df_trust_gov.set_index("Entity").transpose()[country_dict.keys()]
df_trust_journalists = df_trust_journalists.set_index("Entity")[["Trust journalists in this country"]].transpose()[COUNTRY_OWN_LANG_TRUST_JOURN.keys()].rename(columns= COUNTRY_OWN_LANG_TRUST_JOURN)
df_trust_journalists.head(60)

Entity,it,ru,zh,sq,bn,km,hr,el,sv,fi,...,ja,kk,ko,ky,nl,pl,ro,tg,th,mn
Trust journalists in this country,47.4,32.1,61.2,53.8,60.4,73.7,54.0,11.3,70.6,80.4,...,46.1,60.2,27.4,62.0,75.7,66.7,53.0,54.3,69.4,60.0


In [ ]:
from sklearn.cluster import KMeans
# Create dataframe for clustering with covid max cases, pageviews max, and trust in journalists

# Preprocessing the data
DF_ARRAY = np.array(DF.iloc[:,2:5])          # Getting only the numeric features from the dataset
DF_NORM  = preprocessing.normalize(DF_ARRAY) # Normalizing the data

# Creating our Model
kmeans = KMeans(n_clusters = 3)

# Training our model
kmeans.fit(DF_NORM)

In [ ]:
import plotly.graph_objects as go

PLOT = go.Figure()

for C in list(DF.cluster.unique()):
    
    PLOT.add_trace(go.Scatter3d(x = DF[DF.cluster == C]['Annual Income (k$)'],
                                y = DF[DF.cluster == C]['Age'],
                                z = DF[DF.cluster == C]['Spending Score (1-100)'],
                                mode = 'markers', marker_size = 8, marker_line_width = 1,
                                name = 'Cluster ' + str(C)))


### Feature selection for mobility

In [ ]:
data_folder = 'data_2/'

#Download mobility and intervention files
google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')
interventions = pd.read_csv(data_folder+'interventions.csv.zip')

#Download file with covid pageviews #MODIFY WITH THE NEW DATA
file = gzip.open(data_folder+'aggregated_timeseries.json.gz')
data = json.load(file)

In [ ]:
#Change name of the columns
names = ["Retail and Recreations", "Grocery and Pharmacy", "Parks","Transit stations", "Workplace", "Residential"]
google_mobility = google_mobility.rename(dict(zip(google_mobility.columns[8:],names)), axis = 1)

#Transform date string to datetime
google_mobility['date'] = pd.to_datetime(google_mobility['date'])

#Delete columns about region
google_mobility = google_mobility.drop(google_mobility.iloc[:,2:7], axis = 1)

#Keep same dates as for corona wikipedia pages
google_mobility = google_mobility.loc[(google_mobility['date'] < "2020-08-01") & (google_mobility['date'] > "2020-01-21")]

#Regroup all data per country and per date and take the average
mobility = google_mobility.groupby(["country_region_code","country_region","date"]).mean()

google_mobility

In [ ]:
# Calculate total mobility 

google_mobility["total mobility average per day"] = google_mobility[['Retail and Recreations','Grocery and Pharmacy','Parks',
     'Transit stations','Workplace' ,'Residential']].mean(axis=1).reset_index(drop=True) 
     

In [ ]:
google_mobility.head(30)

In [ ]:
mean_mobility = google_mobility.groupby('country_region')["total mobility average per day"].mean()

In [ ]:
mean_mobility.head(60)

In [ ]:
# Plot moblity for the 40 countries 
mobility_countries= pd.DataFrame({"mobility":["mean mobility:"]})
for country in country_dict.items():
    print(str(country[0]))
    if str(country[0])== 'China' or str(country[0])== 'Albania' or str(country[0])== 'Iceland' or str(country[0])== 'Azerbaijan':
        pass
    else:
        output_df = pd.DataFrame({str(country[0]):[mean_mobility[str(country[0])]] })
        mobility_countries = pd.concat([mobility_countries, output_df], axis=1)
mobility_countries

In [ ]:
# x = []
# y = []
# for column in mobility_countries:
#     x.append(mobility_countries[column].values)
#     y.append(column)

mobility_countriesT = mobility_countries.transpose()
mobility_countriesT = mobility_countriesT.drop(index= "mobility", axis=0)
mobility_countriesT = mobility_countriesT.rename(columns= {0: 'mean mobility:'})
# create a new figure
plt.figure(figsize=(40, 10))

# use the scatter() function to plot the points,
# with red circles of size 10
plt.bar(mobility_countriesT.index.values, mobility_countriesT['mean mobility:'].values, color="red")

# add a title
plt.title("Bar Plot of mean mobility per country")

# add x and y axis labels
plt.xlabel("Countries")
plt.ylabel("Mean mobility)")

# show the plot
plt.show()